# Project : Northwind Database
<li><a href="#Load data">Load data to data frames</a></li>
<li><a href="#Inspecting">Inspecting Data</a></li>
<li><a href="#Data Cleaning">Data Cleaning</a></li>
<li><a href="#Exploratory Data Analysis">Exploratory Data Analysis</a></li>
<li><a href="#Save DataFrames to CSV">Save DataFrames to CSV</a></li>

## Load data to data frames
<a id='Load data'></a>

In [3]:
import pandas as pd

In [4]:
# Load the Excel file
excel_name ='Northwind Database.xlsx'

# List all sheets
sheets_name = pd.ExcelFile(excel_name).sheet_names
#Create empty list of data frame names
list_df = []

# Create data frame for all sheets in excel
for sheet in sheets_name:
    globals()[f'df_{sheet.lower()}'] = pd.read_excel(excel_name, sheet_name=sheet)
# Append all data frame names create from excel file
    list_df.append(f'df_{sheet.lower()}')
print(list_df)

['df_orderitem', 'df_orders', 'df_customer', 'df_product', 'df_supplier']


## Inspecting Data
<a id='Inspecting'></a>

In [6]:
# Define function to explore data
def data_exploration(df):
  data ={'name':[],
         'type':[],
         'nulls':[],
         'unique':[],
         'most_frequent':[],
         'per_most_frequent':[]
  }
  for column in df.columns:
    data['name'].append(column)
    data['type'].append(df[column].dtype)
    data['nulls'].append(df[column].isnull().sum())
    data['unique'].append(df[column].nunique())
    data['most_frequent'].append(df[column].value_counts().idxmax())
    data['per_most_frequent'].append(df[column].value_counts().max()/df.shape[0])
  df_1 =pd.DataFrame(data)
  df_1 = df_1.style.format({'per_most_frequent':'{:.2%}'})
  print("Columns",df.shape[1],"\nRows",df.shape[0])
  print("duplicated",df.duplicated().sum())
  return df_1

In [7]:
data_exploration(df_orderitem)

Columns 5 
Rows 2203
duplicated 48


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id,int64,0,2155,2067.000000,0.14%
1,OrderId,int64,0,830,830.000000,1.13%
2,ProductId,int64,0,77,24.000000,2.54%
3,UnitPrice,float64,0,115,18.000000,4.72%
4,Quantity,int64,0,55,20.000000,11.76%


In [8]:
df_orderitem.head()

,Id,OrderId,ProductId,UnitPrice,Quantity
0,683,261,39,18.0,10
1,721,274,35,18.0,3
2,724,275,1,18.0,40
3,738,279,1,18.0,8
4,752,283,76,18.0,50


In [9]:
data_exploration(df_orders)

Columns 5 
Rows 862
duplicated 32


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id&Date,object,0,830,643-1622014,0.35%
1,Month,object,0,12,Apr,12.76%
2,OrderNumber,int64,0,830,543020,0.35%
3,CustomerId,int64,0,89,71,3.71%
4,TotalAmount,float64,0,779,360.000000,0.58%


In [10]:
df_orders.head()

,Id&Date,Month,OrderNumber,CustomerId,TotalAmount
0,1-472012,Jul,542378,85,452.6
1,2-572012,Jul,542379,79,1863.4
2,3-872012,Jul,542380,34,1813.0
3,4-872012,Jul,542381,84,670.8
4,5-972012,Jul,542382,76,3730.0


In [11]:
data_exploration(df_customer)

Columns 6 
Rows 103
duplicated 12


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id,int64,0,91,35,2.91%
1,FirstName,object,0,86,Carlos,3.88%
2,LastName,object,0,90,Wilson,2.91%
3,City,object,0,69,London,6.80%
4,Country,object,0,21,USA,15.53%
5,Phone,object,0,91,(5) 555-1340,2.91%


In [12]:
df_customer.head()

,Id,FirstName,LastName,City,Country,Phone
0,1,Maria,Anders,Berlin,Germany,030-0074321
1,2,Ana,Trujillo,México D.F.,Mexico,(5) 555-4729
2,3,Antonio,Moreno,México D.F.,Mexico,(5) 555-3932
3,4,Thomas,Hardy,London,UK,(171) 555-7788
4,5,Christina,Berglund,Luleå,Sweden,0921-12 34 65


In [13]:
data_exploration(df_product)

Columns 8 
Rows 94
duplicated 16


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id,int64,0,78,19,3.19%
1,ProductName,object,0,78,Teatime Chocolate Biscuits,3.19%
2,Category,object,0,8,Confections,21.28%
3,SupplierId,int64,0,29,3,7.45%
4,UnitPrice,float64,0,63,14.000000,5.32%
5,UnitCost,float64,0,75,10.640000,3.19%
6,Package,object,0,70,24 - 12 oz bottles,4.26%
7,IsDiscontinued,bool,0,2,False,91.49%


In [14]:
df_product.head()

,Id,ProductName,Category,SupplierId,UnitPrice,UnitCost,Package,IsDiscontinued
0,1,Chai,Beverages,1,18.0,13.86,10 boxes x 20 bags,False
1,2,Chang,Beverages,1,19.0,15.39,24 - 12 oz bottles,False
2,3,Aniseed Syrup,Condiments,1,10.0,7.70,12 - 550 ml bottles,False
3,4,Chef Anton's Cajun Seasoning,Condiments,2,22.0,16.50,48 - 6 oz jars,False
4,4,Chef Anton's Cajun Seasoning,Condiments,2,22.0,16.50,48 - 6 oz jars,False


In [15]:
data_exploration(df_supplier)

Columns 8 
Rows 35
duplicated 6


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id,int64,0,29,22,8.57%
1,CompanyName,object,0,29,Zaanse Snoepfabriek,8.57%
2,ContactName,object,0,29,Dirk Luchte,8.57%
3,ContactTitle,object,34,1,,2.86%
4,City,object,0,29,Zaandam,8.57%
5,Country,object,0,16,USA,14.29%
6,Phone,object,0,29,(12345) 1212,8.57%
7,Fax,object,20,13,(12345) 1210,8.57%


In [16]:
df_supplier.head()

,Id,CompanyName,ContactName,ContactTitle,City,Country,Phone,Fax
0,1,Exotic Liquids,Charlotte Cooper,,London,UK,(171) 555-2222,NaN
1,2,New Orleans Cajun Delights,Shelley Burke,NaN,New Orleans,USA,(100) 555-4822,NaN
2,3,Grandma Kelly's Homestead,Regina Murphy,NaN,Ann Arbor,USA,(313) 555-5735,(313) 555-3349
3,4,Tokyo Traders,Yoshi Nagase,NaN,Tokyo,Japan,(03) 3555-5011,NaN
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,NaN,Oviedo,Spain,(98) 598 76 54,NaN


## Data Cleaning
<a id='Data Cleaning'></a>

### OrderItem 

In [19]:
# drop duplicate rows
df_orderitem  = df_orderitem.drop_duplicates()

### Orders

In [21]:
# drop duplicate rows
df_orders = df_orders.drop_duplicates()

In [22]:
# Split the id column by "-" and expand into separate columns
df_orders[['Id&Date', 'Date']] = df_orders['Id&Date'].str.split('-', expand=True)

# Rename column
df_orders = df_orders.rename(columns={'Id&Date': 'Id'})

In [23]:
# Change column type
df_orders['Date'] = df_orders['Date'].astype(int)

# get max to check date order
df_orders['Date'].max()

31122013

In [24]:
# Change column type
df_orders['Date'] = df_orders['Date'].astype(str)

# Dictionary to map month
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Convert 'Month' to numeric values using the dictionary
df_orders['month_no'] = df_orders['Month'].map(month_mapping)

In [25]:
# Concatenate month_no and year
df_orders['month_year'] = df_orders['month_no'].astype(str) + df_orders['Date'].str[-4:]

# Create a column for day
df_orders['day'] = df_orders.apply(lambda x: x['Date'][:len(x['Date']) - len(x['month_year'])], axis=1)

# Convert month to 2 digits 
df_orders['month_year'] = df_orders['month_year'].apply(lambda x: x.zfill(6))

# Convert day to 2 digits
df_orders['day'] = df_orders['day'].apply(lambda x: x.zfill(2))

# Rename column
df_orders = df_orders.rename(columns={'Month': 'OrderDate'})

# Concatenate day and month_year
df_orders['OrderDate'] = df_orders['day'] + df_orders['month_year']

In [26]:
# Convert 'DateString' to proper datetime format
df_orders['OrderDate'] = pd.to_datetime(df_orders['OrderDate'], format='%d%m%Y')

# Format date to mm/dd/yyyy
df_orders['OrderDate'] = df_orders['OrderDate'].dt.strftime('%m/%d/%Y')

# drop columns
df_orders = df_orders.drop(columns=['Date','month_no', 'month_year', 'day'])

### Customer

In [28]:
# drop duplicate rows
df_customer = df_customer.drop_duplicates()

### Product

In [30]:
# Convert False/True to 0/1 by chaning data type to intger
df_product['IsDiscontinued'] = df_product['IsDiscontinued'].astype(int)

# drop duplicate rows
df_product = df_product.drop_duplicates()

### Supplier

In [32]:
# drop column 
df_supplier = df_supplier.drop(columns=['ContactTitle'])

# drop duplicate rows
df_supplier = df_supplier.drop_duplicates()

#Remove Commas
df_supplier = df_supplier.replace(",", "", regex=True)

## Exploratory Data Analysis
<a id='xploratory Data Analysis'></a>

In [34]:
# Create data frames to get old price 
Product_price_list = df_product.drop(columns=['ProductName','SupplierId','UnitCost', 'Package', 'IsDiscontinued','Category'])
orders_price_list = df_orderitem.drop(columns=['Id'])

# join to data frames
price_list = pd.merge(Product_price_list ,orders_price_list , left_on='Id' ,right_on='ProductId',how='right')
price_list['diff'] = price_list['UnitPrice_x']-price_list['UnitPrice_y']

# filter data frame
price_list = price_list[price_list['diff'] != 0]

# drop columns
price_list = price_list.drop(columns=['UnitPrice_x','ProductId','diff'])

# Rename column
price_list = price_list.rename(columns={'UnitPrice_y': 'OldPrice'})

In [35]:
# Change column type
df_orders['Id'] = df_orders['Id'].astype(int)

# Tester 
test = (price_list['OldPrice']*price_list['Quantity']).sum() 

In [36]:
# Making data frame to get all orders with old price
orderId_oldprice = price_list.drop(columns = ['Id','OldPrice','Quantity'])

# drop duplicate rows
orderId_oldprice = orderId_oldprice.drop_duplicates()
orderId_oldprice.describe()

,OrderId
count,250.000000
mean,125.500000
std,72.312977
min,1.000000
25%,63.250000
50%,125.500000
75%,187.750000
max,250.000000


In [37]:
# Tester
test2 = df_orders[df_orders['Id']<= 250]['TotalAmount'].sum()
# check test
test == test2

True

In [38]:
# Add  new column price version
df_orders['PriceVersion'] = df_orders['Id'].apply(lambda x: 'Old' if x <= 250 else 'New')

In [39]:
# drop columns
price_list = price_list.drop(columns=['OrderId','Quantity'])

# drop duplicate rows
price_list = price_list.drop_duplicates()

# Add column with old price to df_product data frame
df_product =  pd.merge(df_product ,price_list , on='Id' ,how='left')

# Rename column
df_product = df_product.rename(columns={'UnitPrice': 'NewPrice'})

data_exploration(df_product)

Columns 9 
Rows 78
duplicated 0


,name,type,nulls,unique,most_frequent,per_most_frequent
0,Id,int64,0,78,1,1.28%
1,ProductName,object,0,78,Chai,1.28%
2,Category,object,0,8,Confections,17.95%
3,SupplierId,int64,0,29,12,6.41%
4,NewPrice,float64,0,63,18.000000,5.13%
5,UnitCost,float64,0,75,7.700000,2.56%
6,Package,object,0,70,24 - 12 oz bottles,5.13%
7,IsDiscontinued,int32,0,2,0,89.74%
8,OldPrice,float64,1,61,14.400000,5.13%


In [40]:
# Replace NaN values with 0 in old price
df_product['OldPrice'] = df_product['OldPrice'].fillna(0)

## Save DataFrames to CSV
<a id='Save DataFrames to CSV'></a>

In [42]:
# Save each DataFrame to CSV with modified filenames
for name in list_df:
    # Use globals() to get the DataFrame by its name
    df = globals()[name]
    
    # Create the filename by removing the first 3 characters from the name
    filename = name[3:] + '.csv'
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False,encoding='utf-16')   
    print(f"Saved DataFrame as '{filename}'")

Saved DataFrame as 'orderitem.csv'
Saved DataFrame as 'orders.csv'
Saved DataFrame as 'customer.csv'
Saved DataFrame as 'product.csv'
Saved DataFrame as 'supplier.csv'
